In [ ]:
pip install gradio

In [2]:
import gradio as gr
import torch
import torchvision.transforms as T
from torchvision.models import vgg19
from PIL import Image
import requests
from io import BytesIO
import os

In [1]:
# Load a pre-trained VGG19 model
model = vgg19(pretrained=True).features
for param in model.parameters():
    param.requires_grad_(False)
model.eval()

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [21]:
import numpy as np
import torch

# Define the style transfer function
def style_transfer(input_image, style):
    # Define image transformations
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.Lambda(lambda x: x.unsqueeze(0)),
    ])
    device = 'cpu'
    
    # # Apply transformations
    # print(type(input_image))
    # print(input_image.shape)
    input_image = Image.fromarray(input_image)
    input_tensor = transform(input_image).to(device)

    # Style image URL dictionary
    style_urls = {
        "Monet": "https://4.bp.blogspot.com/-7p47RxU6vjE/UoyAGx7WTxI/AAAAAAAApyY/tiECyoT6okU/s1600/1899-1900+The+Iris+Garden+at+Giverny+oil+on+canvas+Yale+89.5+x+92.1+cm+University+Art+Gallery%252C+New+Haven+CT.jpg",
        "sakura": "https://i2.kknews.cc/SIG=1cjibd3/ctp-vzntr/1528451006916p490op5297.jpg",
        "sky": "https://tse2.mm.bing.net/th?id=OIP.g0-Xd7agO_cy-P0zAeIA-gHaEo&pid=Api&P=0&h=180",
    }

    # Load and transform the style image
    style_image = Image.open(requests.get(style_urls[style], stream=True).raw)
    style_tensor = transform(style_image).to(device)

    w_min = min(input_tensor.shape[2], style_tensor.shape[2])
    h_min = min(input_tensor.shape[3], style_tensor.shape[3])
    input_tensor = input_tensor[:, :, :w_min, :h_min]
    style_tensor = style_tensor[:, :, :w_min, :h_min]

    print(style_tensor.shape, input_tensor.shape)

    # # Simple style transfer (this is a placeholder for an actual style transfer algorithm)
    # with torch.no_grad():
    #     styled_tensor = model(input_tensor + style_tensor)  # Simulate a combination of features
    styled_tensor = input_tensor * 0.7 + style_tensor *0.4

    print(styled_tensor.shape)
    # # Remove the normalization to display the image correctly
    styled_tensor = styled_tensor.squeeze(0).cpu().detach()
    styled_tensor = styled_tensor.mul(torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)).add(torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1))
    styled_image = T.ToPILImage()(styled_tensor)

    return styled_image
